In [1]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_hdf('data/train_preprocessed_v4.h5', low_memory = False)
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid
0,7.5,-73.978463,40.759072,-73.963577,40.774255,1,2014.0,9,6,15,1.719640,21.029044,0
1,6.5,-73.982397,40.770245,-73.972450,40.764062,1,2012.0,5,1,7,1.122217,21.940741,0
2,12.9,-74.006507,40.718808,-73.961096,40.760711,1,2009.0,5,2,20,5.210553,20.679895,0
3,20.1,-73.999242,40.734165,-73.972597,40.751228,2,2010.0,7,4,22,3.008668,21.891888,0
4,4.1,-73.977750,40.746274,-73.972760,40.755768,1,2010.0,8,2,13,0.626738,21.931951,0


In [13]:
df_test = pd.read_hdf('data/test_prepro_v4.h5', low_memory = False)
df_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance,JFK_distance,invalid,key
0,-73.972484,40.742743,-73.918937,40.764496,1,2009.0,1,4,0,5.991612,16.117319,0,2009-01-01 00:01:04.0000003
1,-73.985850,40.722826,-73.986301,40.739347,1,2009.0,1,4,0,0.509264,23.237424,0,2009-01-01 00:01:26.0000001
2,-73.988917,40.740142,-73.982769,40.777291,1,2009.0,1,4,0,1.328903,23.142153,0,2009-01-01 00:04:42.0000001
3,-73.977163,40.764490,-73.914474,40.771575,1,2009.0,1,4,0,6.974103,15.690509,0,2009-01-01 00:04:54.0000001
4,-73.948849,40.778003,-73.977678,40.748692,2,2009.0,1,4,0,3.329684,19.450750,0,2009-01-01 00:04:59.0000004


In [14]:
df_test.drop(columns = ['passenger_count'], inplace = True)
df.drop(columns = ['passenger_count'], inplace = True)

In [5]:
def make_long(long):
    long = long + 72.8
    return long
long_vec = np.vectorize(make_long)

def make_lat(lat):
    lat = lat - 40.5
    return lat
lat_vec = np.vectorize(make_lat)

In [6]:
def scale_lat_long(df):
    df['pickup_latitude'] = lat_vec(df.pickup_latitude)
    df['pickup_longitude'] = long_vec(df.pickup_longitude)
    df['dropoff_latitude'] = lat_vec(df['dropoff_latitude'])
    df['dropoff_longitude'] = long_vec(df.dropoff_longitude)
    return df

In [15]:
df_test.drop(columns = ['weekday'], inplace = True)
df.drop(columns = ['weekday'], inplace = True)

In [16]:
def det_dist(dist, inv):
    if(dist >= 50):
        return 1
    return inv
dis_vec = np.vectorize(det_dist)
df['invalid'] = dis_vec(df.distance, df.invalid)
df_test['invalid'] = dis_vec(df_test.distance, df_test.invalid)

In [17]:
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,distance,JFK_distance,invalid
0,7.5,-73.978463,40.759072,-73.963577,40.774255,2014.0,9,15,1.719640,21.029044,0
1,6.5,-73.982397,40.770245,-73.972450,40.764062,2012.0,5,7,1.122217,21.940741,0
2,12.9,-74.006507,40.718808,-73.961096,40.760711,2009.0,5,20,5.210553,20.679895,0
3,20.1,-73.999242,40.734165,-73.972597,40.751228,2010.0,7,22,3.008668,21.891888,0
4,4.1,-73.977750,40.746274,-73.972760,40.755768,2010.0,8,13,0.626738,21.931951,0


In [9]:
import lightgbm as lgb

params = {
        'learning_rate': 0.70,
        'application': 'regression',
        'max_depth': ,
        'num_leaves': 200,
        'verbosity': -1,
        'metric': 'RMSE',
    }

In [18]:
from sklearn.model_selection import train_test_split
y = df['fare_amount']
X = df.drop(columns=['fare_amount'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [19]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=500,
                early_stopping_rounds = 10)
gbm.save_model('model_lim_features_v2.txt')

[1]	valid_0's rmse: 41.546
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 41.4471
[3]	valid_0's rmse: 41.416
[4]	valid_0's rmse: 41.4018
[5]	valid_0's rmse: 41.3916
[6]	valid_0's rmse: 41.3856
[7]	valid_0's rmse: 41.3813
[8]	valid_0's rmse: 41.3778
[9]	valid_0's rmse: 41.3707
[10]	valid_0's rmse: 41.3671
[11]	valid_0's rmse: 41.3651
[12]	valid_0's rmse: 41.3628
[13]	valid_0's rmse: 41.3605
[14]	valid_0's rmse: 41.3587
[15]	valid_0's rmse: 41.3564
[16]	valid_0's rmse: 41.3553
[17]	valid_0's rmse: 41.3521
[18]	valid_0's rmse: 41.3498
[19]	valid_0's rmse: 41.3483
[20]	valid_0's rmse: 41.3467
[21]	valid_0's rmse: 41.3454
[22]	valid_0's rmse: 41.3422
[23]	valid_0's rmse: 41.3384
[24]	valid_0's rmse: 41.3378
[25]	valid_0's rmse: 41.3341
[26]	valid_0's rmse: 41.3335
[27]	valid_0's rmse: 41.3326
[28]	valid_0's rmse: 41.332
[29]	valid_0's rmse: 41.3293
[30]	valid_0's rmse: 41.3286
[31]	valid_0's rmse: 41.3283
[32]	valid_0's rmse: 41.3263
[33]	valid_0's rmse: 41

[277]	valid_0's rmse: 41.2871
[278]	valid_0's rmse: 41.2871
[279]	valid_0's rmse: 41.2871
[280]	valid_0's rmse: 41.287
[281]	valid_0's rmse: 41.287
[282]	valid_0's rmse: 41.2869
[283]	valid_0's rmse: 41.2869
[284]	valid_0's rmse: 41.2868
[285]	valid_0's rmse: 41.2868
[286]	valid_0's rmse: 41.2868
[287]	valid_0's rmse: 41.2867
[288]	valid_0's rmse: 41.2867
[289]	valid_0's rmse: 41.2867
[290]	valid_0's rmse: 41.2866
[291]	valid_0's rmse: 41.2866
[292]	valid_0's rmse: 41.2865
[293]	valid_0's rmse: 41.2865
[294]	valid_0's rmse: 41.2864
[295]	valid_0's rmse: 41.2864
[296]	valid_0's rmse: 41.2863
[297]	valid_0's rmse: 41.2863
[298]	valid_0's rmse: 41.2863
[299]	valid_0's rmse: 41.2862
[300]	valid_0's rmse: 41.2862
[301]	valid_0's rmse: 41.2862
[302]	valid_0's rmse: 41.2862
[303]	valid_0's rmse: 41.2862
[304]	valid_0's rmse: 41.2861
[305]	valid_0's rmse: 41.286
[306]	valid_0's rmse: 41.286
[307]	valid_0's rmse: 41.286
[308]	valid_0's rmse: 41.286
[309]	valid_0's rmse: 41.2859
[310]	valid_0's 

In [20]:
keys = df_test['key'] 
df_test.drop(columns = ['key'], inplace = True)
df_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,distance,JFK_distance,invalid
0,-73.972484,40.742743,-73.918937,40.764496,2009.0,1,0,5.991612,16.117319,0
1,-73.985850,40.722826,-73.986301,40.739347,2009.0,1,0,0.509264,23.237424,0
2,-73.988917,40.740142,-73.982769,40.777291,2009.0,1,0,1.328903,23.142153,0
3,-73.977163,40.764490,-73.914474,40.771575,2009.0,1,0,6.974103,15.690509,0
4,-73.948849,40.778003,-73.977678,40.748692,2009.0,1,0,3.329684,19.450750,0


In [21]:
pred_fares = gbm.predict(df_test, num_iteration=gbm.best_iteration)
df_final = pd.DataFrame({'key':keys, 'fare_amount':pred_fares})

In [22]:
df_final.to_csv(r'predictions/lightgbm_withfeaures_v7.csv', index = False)